<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/CNN_model_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to values between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape data to fit model input shape (28, 28, 1)
x_train_cnn = x_train.reshape((60000, 28, 28, 1))
x_test_cnn = x_test.reshape((10000, 28, 28, 1))

# Create an instance of ImageDataGenerator with augmentation options
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
)
datagen.fit(x_train_cnn)

# Create a sequential model
model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  # Regularization
model_cnn.add(layers.Dropout(0.5))  # Dropout regularization
model_cnn.add(layers.Dense(10, activation='softmax'))

# Compile the model
model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with augmented data
model_cnn.fit(datagen.flow(x_train_cnn, y_train, batch_size=32), epochs=5, validation_data=(x_test_cnn, y_test))

# Evaluate the model
test_loss_cnn, test_acc_cnn = model_cnn.evaluate(x_test_cnn, y_test)
print(f'Test accuracy: {test_acc_cnn}')

# Make predictions
predictions_cnn = model_cnn.predict(x_test_cnn)

# Plotting some test images with their predicted labels
import matplotlib.pyplot as plt

for i in range(5):
    plt.imshow(x_test_cnn[i].reshape(28, 28), cmap='gray')
    plt.title(f'Predicted label: {np.argmax(predictions_cnn[i])}')
    plt.axis('off')
    plt.show()